# Orchestral AI Demo

This notebook shows how to use Orchestral AI to make an AI agent that can interact with the SBND electronic logbook.

In [2]:
from orchestral import Agent
from orchestral.llm import *
from orchestral import define_tool
from tool_logger import ToolCallLogger


In [3]:
with open('pwd.txt', 'r') as f:
    ecl_pwd = f.read().strip()  


In [4]:
@define_tool()
def ecl_search(category='Shift', limit=3, after=None):
    """
    Searches the SBND electronic logbook (ECL) for all the first `limit` entries of a certain `category`.

    Args:
        category: The ECL category
        limit: Limit to the number of entries to retrieve
        after (str): Searches for entries after a certain date. The date has to be in the following formats:
            <n>days (ex: "1days" for the last 24h entries)
            <n>hours (ex: "1hours" for the last hour entries)
            <n>minutes (ex: "1minutes" for the last minute entries)
            yyyy-mm-dd+hh:mm:ss (ex: "2012-04-01+12:00:00"

    Returns:
        List of entries

    NOTE: If a category is not provide, use "Shift" as default.
    """
    from ecl_api import ECL, ECLEntry
    import xml.etree.ElementTree as ET
    
    password = ecl_pwd
    url = "https://dbweb1.fnal.gov:8443/ECL/sbnd/E"

    ecl = ECL(url=url, user='sbndprm', password=password)

    # return [ecl.get_entry(entry_id=7252)]
    text = ecl.search(category=category, limit=limit, after=after)

    xml = ET.fromstring(text)
    entries = xml.findall('./entry')
    result = []
    for entry in entries:
        result.append((entry.attrib, entry.tag))
    return result

In [5]:
hooks = [ToolCallLogger(
        verbose=True,       # Show full arguments (False = just tool names)
        show_results=False  # Show tool outputs (can be very verbose)
    )]

In [6]:
agent = Agent(
    llm=Ollama(),
    # llm=GPT(model='gpt-5'),
    tools=[ecl_search],
    tool_hooks=hooks,
    system_prompt="You are a helpful physics assistant with knowledge on how to retrive entries from the logbook."
)


In [23]:
response = agent.run('What is the ecl_search function?')
print(response.text)

The ecl_search function is a tool for querying the SBND electronic logbook (ECL). It returns a list of log entries matching your criteria.

Inputs
- category: ECL category to search (string). If omitted, it defaults to "Shift".
- limit: Maximum number of entries to return (integer).
- after: Time filter (string) to only return entries after a given time. Accepted formats:
  - Relative: "<n>days", "<n>hours", "<n>minutes" (e.g., "2days", "1hours").
  - Absolute: "yyyy-mm-dd+hh:mm:ss" (e.g., "2024-01-15+08:00:00").

Output
- A list of matching ECL entries (typically the most recent up to the given limit).

Example calls
- Get the last 10 Shift entries from the past day:
  - ecl_search({ category: "Shift", limit: 10, after: "1days" })
- Get 5 DAQ entries since a specific timestamp:
  - ecl_search({ category: "DAQ", limit: 5, after: "2024-10-01+00:00:00" })
- Get the last 3 entries in the default category ("Shift"):
  - ecl_search({ limit: 3 })


In [24]:
response = agent.run('Can you fetch the entries form the last 24 hours in category Shift?')
print(response.text)


>>> TOOL CALL: ecl_search
    Arguments: {
  "category": "Shift",
  "limit": 200,
  "after": "1days"
}
<<< ecl_search completed
Here are the Shift category entries from the last 24 hours (41 total), newest first:

- [32671] 01/28/2026 17:30:05 — csauer — Beam Checklist - v2 (images: 0, files: 0)
- [32670] 01/28/2026 17:04:37 — csauer — Shifter Checklist - Run2 - v6.2 (images: 2, files: 0)
- [32669] 01/28/2026 16:31:19 — csauer — Beam Checklist - v2 (images: 0, files: 0)
- [32668] 01/28/2026 15:58:47 — csauer — Start of Shift Checklist - v4 (images: 0, files: 0)
- [32667] 01/28/2026 15:58:32 — pmachado — End of Shift Checklist - v3 (images: 0, files: 0)
- [32666] 01/28/2026 15:27:11 — pmachado — Beam Checklist - v2 (images: 0, files: 0)
- [32665] 01/28/2026 15:05:47 — pmachado — Shifter Checklist - Run2 - v6.2 (images: 2, files: 0)
- [32664] 01/28/2026 14:29:30 — pmachado — Beam Checklist - v2 (images: 0, files: 0)
- [32663] 01/28/2026 13:29:44 — pmachado — Beam Checklist - v2 (images:

In [7]:
response = agent.run('Can you fetch the last 10 entries and give me the result?')
print(response.text)


>>> TOOL CALL: ecl_search
    Arguments: {
  "category": "General",
  "limit": 10
}
<<< ecl_search completed
Here are the 10 most recent **General**‑category entries from the SBND Electronic Logbook:

| # | Entry ID | Author | Subject | Timestamp | Form | Images | Files |
|---|----------|--------|---------|-----------|------|--------|-------|
| 1 | 29697 | **mnebot** |  | 11/27/2025 08:21:09 | Start of Shift Checklist – v4 | 0 | 0 |
| 2 | 29401 | **lupis** | Digital Noise | 11/21/2025 02:33:24 | default | 3 | 0 |
| 3 | 28727 | **fnicolas** | Large temperature change in rack temperatures (>4 °C) | 11/01/2025 14:45:17 | default | 5 | 0 |
| 4 | 25719 | **manoa** |  | 08/04/2025 19:43:40 | Alarm Acknowledged | 0 | 0 |
| 5 | 25412 | **wforeman** |  | 07/22/2025 13:53:10 | Alarm Acknowledged | 1 | 0 |
| 6 | 24526 | **gfricano** |  | 06/28/2025 17:02:16 | Alarm Acknowledged | 1 | 0 |
| 7 | 24365 | **prabhjot** |  | 06/25/2025 23:47:46 | Alarm Acknowledged | 1 | 0 |
| 8 | 22111 | **amoor** | 

In [8]:
response = agent.run('Please use Shift as category')
print(response.text)


>>> TOOL CALL: ecl_search
    Arguments: {
  "category": "Shift",
  "limit": 10
}
<<< ecl_search completed
Here are the **10 most recent “Shift”** entries from the SBND Electronic Logbook (all dates in local US/Central time):

| # | Entry ID | Author | Subject | Timestamp | Form | Images | Files |
|---|----------|--------|---------|-----------|------|--------|-------|
| 1 | 32658 | **mdeltutt** | – | 01/28/2026 10:34:12 | Beam Checklist – v2 | 0 | 0 |
| 2 | 32657 | **mdeltutt** | – | 01/28/2026 09:36:53 | Beam Checklist – v2 | 0 | 0 |
| 3 | 32656 | **mdeltutt** | – | 01/28/2026 08:57:44 | Shifter Checklist – Run2 – v6.2 | 2 | 0 |
| 4 | 32655 | **mdeltutt** | – | 01/28/2026 08:17:25 | Beam Checklist – v2 | 0 | 0 |
| 5 | 32654 | **mdeltutt** | – | 01/28/2026 07:59:27 | Start of Shift Checklist – v4 | 0 | 0 |
| 6 | 32653 | **mnebot** | – | 01/28/2026 07:43:32 | End of Shift Checklist – v3 | 0 | 0 |
| 7 | 32652 | **mnebot** | – | 01/28/2026 07:31:05 | Beam Checklist – v2 | 0 | 0 |
| 8 | 3